In [1]:
import requests
import xml.etree.ElementTree as ET

# boto3 not installed by default in anaconda (conda install boto3)
import boto3
import boto3.session
import pandas as pd
import io
from jwt import JWT
from botocore.client import Config

class SimvaBrowser(object):
    def __init__(self, auth, storage_url, accept='.json', ca_file=None, bucket_name='traces', delimiter='/'):
        self.auth = auth      
        self.storage_url = storage_url;
        self.accept='.json';
        self.ca_file = ca_file;
        self.bucket_name = bucket_name;
        self.delimiter = delimiter;
        jwt_parser = JWT()
        self.access_token = jwt_parser.decode(self.auth.access_token, do_verify=False)

        self._storage_login();
        
        self.base_path = 'users' + self.delimiter + self.access_token['preferred_username'] + self.delimiter;
        self.current_path = self.base_path;

        self._update_files()
        
        self.layoutButtons = widgets.Layout(width='auto')
        self.buttonRun = widgets.Button(description='Run analyse', button_style='success', visible=False,
                                        layout=self.layoutButtons)
        self.output=widgets.Output(layout={'border': '1px solid black'})

    def _storage_login(self):
        data = {
            'Action':'AssumeRoleWithWebIdentity',
            'Version': '2011-06-15',
            'DurationSeconds' : 3600,
            'WebIdentityToken': self.auth.access_token
        };
        response = requests.post(self.storage_url, data=data, verify=self.ca_file)
        if response.status_code != 200:
            print('Problems getting temporary credentials')
            print(response.text)
        else:
            ns = {'sts' : 'https://sts.amazonaws.com/doc/2011-06-15/'}
            root = ET.fromstring(response.text)
            credentials = root.find('sts:AssumeRoleWithWebIdentityResult', ns).find('sts:Credentials', ns)
            self.access_key_id = credentials.find('sts:AccessKeyId', ns).text
            self.secret_access_key = credentials.find('sts:SecretAccessKey', ns).text
            self.session_token = credentials.find('sts:SessionToken', ns).text
        
    def _s3_client(self):
        aws_session=boto3.session.Session(aws_access_key_id=self.access_key_id,
                                    aws_secret_access_key=self.secret_access_key,
                                    aws_session_token=self.session_token,
                                    region_name='us-east-1')
        return aws_session.client(service_name='s3'
                            , endpoint_url=self.storage_url
                            , verify=self.ca_file)

    def _list_files(self, path):
        s3_client = self._s3_client();
        folder = s3_client.list_objects(Bucket=self.bucket_name
            , Prefix=path
            , Delimiter=self.delimiter)
        files = list()
        contents = folder.get('Contents')
        if contents:
            for o in contents:
                files.append(o.get('Key')[len(self.current_path):])
        return files

    def _list_folders(self, path):
        s3_client = self._s3_client();
        folder = s3_client.list_objects(Bucket=self.bucket_name
            , Prefix=path
            , Delimiter=self.delimiter)
        folders = list()
        contents = folder.get('CommonPrefixes')
        if contents:
            for o in contents:
                folders.append(o.get('Prefix')[len(self.current_path):])
        return folders

    def get_file_content(self, path):
        s3_client = self._s3_client();
        file = s3_client.get_object(Bucket=self.bucket_name, Key=path)
        return file['Body'].read()

    def _isdir(self, path):
        return path[len(path)-1] == self.delimiter;

    
    def _update_files(self):
        self.files = list()
        self.dirs = list()
        if(self._isdir(self.current_path)):
            self.dirs = self._list_folders(self.current_path)
            self.files = self._list_files(self.current_path)

    def widget(self):
        box = widgets.VBox()
        self._update(box)
        return box

    def _update(self, box):
        def on_click(b):
            if b.description == '..':
                if len(self.current_path) > len(self.base_path):
                    self.current_path = self.current_path.rpartition(self.delimiter)[0].rpartition(self.delimiter)[0] + self.delimiter
                else:
                    self.current_path = self.base_path
                with self.output:
                    clear_output()
            else:
                self.current_path = self.current_path + b.description
            self._update_files()
            self._update(box)

        buttons = []
        button = widgets.Button(description='..', background_color='#d0d0ff', layout=self.layoutButtons)
        button.on_click(on_click)
        buttons.append(button)
        if(self._isdir(self.current_path)):
            for f in self.dirs:
                button = widgets.Button(description=f, background_color='#d0d0ff', layout=self.layoutButtons)
                button.on_click(on_click)
                buttons.append(button)
            for f in self.files:
                fileExtension = os.path.splitext(f)[1]
                if fileExtension == self.accept:
                    button = widgets.Button(description=f, button_style='success', layout=self.layoutButtons)
                    button.on_click(on_click)
                    buttons.append(button)
        else:
            self.buttonRun.visible=True
            buttons.append(self.buttonRun)
            self.output.clear_output(wait=False)
            with self.output:
                print("Click to run analyse to the file")
            buttons.append(self.output)
        box.children = tuple([widgets.HTML("<h3>%s</h3>" % (self.current_path,))] + buttons)